In [1]:
import gurobipy as gurobi
import pandas as pd
import numpy as np

In [2]:
def compute_result_month_based(*args):
    m, products_plan, transports_plan, df_request, countries, products_costs, transports_cost, products, capacities, prod_sol, transport_sol = args
    # MONTHLY REQUEST
    df_month_request = df_request.loc[df_request["Month"] == f"{m}2004"]
    requests = {}
    for index, row in df_month_request.iterrows():
        c = row["Country"]
        p = row["Product"]
        r = row["Quantity"]
        requests[f"{c},{p}"] = r

    model = gurobi.Model(f"{m}")
    # PIANO DI PRODUZIONE
    g_production = {}
    
    for key in products_plan:
        g_production[key] = model.addVar(name = key, lb=0, vtype = gurobi.GRB.INTEGER)

    # PIANO DI TRASPORTO
    g_transport = {}
    
    for key in transports_plan:
        g_transport[key] = model.addVar(name = key, lb = 0, vtype = gurobi.GRB.INTEGER)
    
    # VINCOLI DI CAPACITA'
    capacity_constrs = {}
    for c in countries:
        capacity_constrs[f"country : {c} - CAPACITY CONSTRAINT"] = model.addConstr(gurobi.quicksum(g_production[f"{c},{p}"] for p in products) <= capacities[c], name = f"country : {c} - CAPACITY CONSTRAINT") # capacity constraint 

    # VINCOLI DI DOMANDA
    requests_constrs = {}  
    for p in products:
        for c in countries:
            
            # EXPORT COUNTRIES
            export_countries = []
            for d in countries:
                if f"{c},{d},{p}" in transports_plan:
                    export_countries.append(f"{c},{d},{p}")
            # IMPORT COUNTRIES
            import_countries = []
            for o in countries:
                if f"{o},{c},{p}" in transports_plan:
                    import_countries.append(f"{o},{c},{p}")
            
            constr = model.addConstr((g_production[f"{c},{p}"] - gurobi.quicksum(g_transport[key] for key in export_countries) + gurobi.quicksum(g_transport[key] for key in import_countries)) >= (requests[f"{c},{p}"]), name = f"country : {c} - product : {p} - REQUEST CONSTRAINT")
            requests_constrs[f"country : {c} - product : {p} - REQUEST CONSTRAINT"] = constr
    
    # VINCOLI DI FLUSSO
    flux_constrs = {}
    for p in products:
        for c in countries:
            
            # EXPORT COUNTRIES
            export_countries = []
            for d in countries:
                if f"{c},{d},{p}" in transports_plan:
                    export_countries.append(f"{c},{d},{p}")
            # IMPORT COUNTRIES
            import_countries = []
            for d in countries:
                if f"{d},{c},{p}" in transports_plan:
                    import_countries.append(f"{d},{c},{p}")
            
            constr = model.addConstr((g_production[f"{c},{p}"] - gurobi.quicksum(g_transport[key] for key in export_countries) + gurobi.quicksum(g_transport[key] for key in import_countries)) >= 0, name = " {c} FLUX CONSTRAINT")

    production_objective = gurobi.quicksum(products_costs[key]*g_production[key] for key in products_plan)
    transport_objective = gurobi.quicksum(transports_cost[",".join(key.split(",")[:2])]*g_transport[key] for key in transports_plan)
    model.setObjective(production_objective + transport_objective, gurobi.GRB.MINIMIZE) 
    model.optimize()
    
    print(f"{m}")
    """
    model.computeIIS()
    
    for ci in model.getConstrs():
        if ci.getAttr("IISConstr") == 1:
            print(f"{ci.ConstrName} : {ci.getAttr("IISConstr")}")
    """
    if model.status == gurobi.GRB.OPTIMAL:
        print(f"Soluzione ottima trovata:")
        for key, var in g_production.items():
            new_key = key+f"{m}2004"
            print(new_key)
            prod_sol[new_key] = var.X
        for key, var in g_transport.items():
            new_key = key+f"{m}2004"
            transport_sol[new_key] = var.X
        print(f"Valore ottimo: {model.ObjVal}")
    else:
        print("Nessuna soluzione ottima trovata.")
    print()

    return prod_sol, transport_sol

In [3]:
df_request = pd.read_csv('../data/02_input_target.csv')
products = df_request["Product"].unique()
countries = df_request["Country"].unique()

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# COSTI DI PRODUZIONE

df_prod_costs = pd.read_csv('../data/03_input_productionCost.csv')

products_plan = [] # key for the dictionary
products_costs = {}

for index, row in df_prod_costs.iterrows():
    s = row["Country"]
    p = row["Product"]
    c = row["Unit Cost"]
    products_costs[f"{s},{p}"] = c
    products_plan.append(f"{s},{p}")

# COSTI DI TRANSPORTO

df_transport_costs = pd.read_csv('../data/02_03_input_shipmentsCost.csv')

transports_plan = [] # key for the dictionary
transports_cost = {}
transport_sol = {}
for index, row in df_transport_costs.iterrows():
    o = row["Origin"]
    d = row["Destination"]
    c = row["Unit Cost"]  
    transports_cost[f"{o},{d}"] = c
    transports_cost[f"{d},{o}"] = c
    for p in products: 
        transports_plan.append(f"{o},{d},{p}")
        transports_plan.append(f"{d},{o},{p}")
        for m in months:
            transport_sol[f"{o},{d},{p},{m}2004"] = 0
            transport_sol[f"{d},{o},{p},{m}2004"] = 0

# CAPACITA'

df_capacity = pd.read_csv('../data/02_input_capacity.csv')
capacities = {}

for index, row in df_capacity.iterrows():
    s = row["Country"]
    c = row["Monthly Capacity"]
    capacities[s] = c

prod_sol = {f"{c},{p},{m}2004" : 0 for c in countries for p in products for m in months}

In [ ]:
from multiprocessing import Pool

with Pool() as pool:
    risultati = pool.map(compute_result_month_based, [[m, products_plan, transports_plan, df_request, countries, products_costs, transports_cost, products, capacities, prod_sol, transport_sol] for m in months])